In [2]:
import json
import requests
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
import folium
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, MultiSearch, Q
import folium
from folium.plugins import MarkerCluster, HeatMap
import openrouteservice
import time
import itertools as it
import math

es=Elasticsearch(['http://es_01'], timeout=10000,sniff_on_start=True)

In [3]:
#Helper functions
def get_path(start,end):
    time.sleep(2)

    coords = [start , end]
    
    client = openrouteservice.Client(key='5b3ce3597851110001cf6248dfdd2bddd5ec4a9fa8fb0450dcf61599') # Specify your personal API key
    routes = client.directions(coordinates=coords,extra_info=["osmid"])
    return routes
def get_park(name):
    park_shape = Search(using=es,index='*park*').filter('match',GIS__OPENSPACE__NAME__keyword=name.upper())
    return park_shape.execute().to_dict()['hits']['hits'][0]['_source']

def get_park_path(park_shape):
    park_path = Search(using=es,index='*pathways*').filter('geo_shape',path={'shape':park_shape})
    return [i['_source'] for i in park_path.execute().to_dict()['hits']['hits']]

def get_business(name):
    els = Search(using=es,index=['regina-*-businesses','regina-park*']).filter('match',name__keyword=name)
    resp=els.execute()
    return resp.to_dict()['hits']['hits'][0]['_source'], {'type':'Point','coordinates':[resp.to_dict()['hits']['hits'][0]['_source']['geoPoint']['lon'],resp.to_dict()['hits']['hits'][0]['_source']['geoPoint']['lat']]}

def generateBaseMap(default_location=[0,0], default_zoom_start=2,tiles='OpenStreetMap'):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start,tiles=tiles)
    return base_map
def build_map(name1,name2,type1='business',type2='business',zoom=12,tile_type='OpenStreetMap'):
    if type1 in ['park','business'] and type2 in ['park','business']:
        None
    else:
        return "type1 and type2 need to be park or business"
    if type1 == 'park':
        #get park and business data
        park_name = name1
        business_name=name2
        shape1 = get_park(name1)
        path1 = shapely.geometry.mapping(gpd.GeoDataFrame(geometry=[ shapely.geometry.shape(i['path']) for i in get_park_path(shape1['shape'])]).unary_union)
        park_centroid = shapely.geometry.mapping(gpd.GeoDataFrame(geometry=[shapely.geometry.shape(shape1['shape'])]).centroid.geometry.iloc[0])
        business1, business_coords = get_business(name2)
        route1 = openrouteservice.convert.decode_polyline(get_path(business_coords['coordinates'],park_centroid['coordinates'])['routes'][0]['geometry'])
    elif type2 == 'park':
        #get business and park data
        park_name = name2
        business_name=name1
        shape1 = get_park(name2)
        path1 = shapely.geometry.mapping(gpd.GeoDataFrame(geometry=[ shapely.geometry.shape(i['path']) for i in get_park_path(shape1['shape'])]).unary_union)
        park_centroid = shapely.geometry.mapping(gpd.GeoDataFrame(geometry=[shapely.geometry.shape(shape1['shape'])]).centroid.geometry.iloc[0])
        business1, business_coords = get_business(name1)
        route1 = openrouteservice.convert.decode_polyline(get_path(park_centroid['coordinates'],business_coords['coordinates'])['routes'][0]['geometry'])
    else:
        #get 2 business data
        business1, business_coords1 = get_business(name1)
        business2, business_coords2 = get_business(name2)
        route1 = openrouteservice.convert.decode_polyline(get_path(business_coords1['coordinates'],business_coords2['coordinates'])['routes'][0]['geometry'])
        route_centroid = shapely.geometry.mapping(gpd.GeoDataFrame(geometry=[shapely.geometry.shape(route1)]).centroid.geometry.iloc[0])
    if type1 =='park' or type2 == 'park':
        #i business and 1 park
        route = folium.GeoJson(
            route1,
            popup='Route',
            style_function=lambda feature: {
                'fillColor': '#d9534f',
                'color' : '#d9534f',
                'weight' : 5,
                'fillOpacity' : 0.5,
                }
        )
        path = folium.GeoJson(
            shapely.geometry.mapping(gpd.clip(gpd.GeoDataFrame(geometry=[shapely.geometry.shape(path1)]),gpd.GeoDataFrame(geometry=[shapely.geometry.shape(shape1['shape'])])).geometry[0]),
            popup=park_name + ' Path',
            style_function=lambda feature: {
                'fillColor': '#428bca',
                'color' : '#428bca',
                'weight' : 5,
                'fillOpacity' : 0.5,
                }
        )
        park = folium.GeoJson(
            shape1['shape'],
            popup=park_name,
            style_function=lambda feature: {
                'fillColor': '#5cb85c',
                'color' : '#5cb85c',
                'weight' : 5,
                'fillOpacity' : 0.5,
                }
        )
        park_pin = folium.Marker(
            location=[park_centroid['coordinates'][1],park_centroid['coordinates'][0]],
            popup=park_name,
            name='Activity',
            icon=folium.Icon(

                                          icon='tree',    # icon code from above link
                                          prefix='fa'))
        rest_pin=folium.Marker(
            location=[business_coords['coordinates'][1],business_coords['coordinates'][0]],
            popup=business_name,
            icon=folium.Icon(

                                          icon='circle',    # icon code from above link
                                          prefix='fa')
        )

        base_map=generateBaseMap([park_centroid['coordinates'][1],park_centroid['coordinates'][0]],zoom,tiles=tile_type)

        park_pin.add_to(base_map)
        rest_pin.add_to(base_map)

        base_map.add_child(park)
        base_map.add_child(path)
        base_map.add_child(route)
    else:
        #2 businesses
        route = folium.GeoJson(
            route1,
            popup='Route',
            style_function=lambda feature: {
                'fillColor': '#d9534f',
                'color' : '#d9534f',
                'weight' : 5,
                'fillOpacity' : 0.5,
                }
        )
        
        act_pin = folium.Marker(
            location=[business_coords1['coordinates'][1],business_coords1['coordinates'][0]],
            popup=name1,
            name='Activity',
            icon=folium.Icon(

                                          icon='circle',    # icon code from above link
                                          prefix='fa'))
        rest_pin=folium.Marker(
            location=[business_coords2['coordinates'][1],business_coords2['coordinates'][0]],
            popup=name2,
            icon=folium.Icon(

                                          icon='circle',    # icon code from above link
                                          prefix='fa')
        )
        base_map=generateBaseMap([route_centroid['coordinates'][1],route_centroid['coordinates'][0]],zoom,tiles=tile_type)

        act_pin.add_to(base_map)
        rest_pin.add_to(base_map)


        base_map.add_child(route)
    return base_map

# Generate Map
Use the function below to generate html maps. You can use the [map](http://localhost:5601/app/maps#/map/9d82a020-d823-11ea-b29a-b5a7bdd10a20?_g=(filters:!(),refreshInterval:(pause:!f,value:0),time:(from:now-15m,to:now))&amp;_a=(filters:!(),query:(language:kuery,query&_a=(filters:!(),query:(language:kuery,query:''))) here to find business and park names to build your map.
NOTE: this cannot generate 2 park experiences
parameters:
- name1 : the name of the first park or business
- name2 : the name of the second park or business
- type1 : if name1 is a business or park (default business)
- type2 : if name2 is a business or park (default business)
- zoom : the zoom level of the map (default 12) larger is closer
- tile_type : the background tile to use for the map (default OpenStreetMap) see post [here](https://python-visualization.github.io/folium/quickstart.html#Markers) for tile options

In [5]:
base_map = build_map('Rochdale Park',"Gringo's Tacos",type1='park',type2='business', zoom=12,tile_type='OpenStreetMap')

In [6]:
#view map
base_map

In [8]:
base_map.save(outfile= "../outputs/maps/rochdale-tacos.html")

